# Calidad de datos anuales extraidos de NORTE LITORAL

En este notebook se describen los procesos necesarios para comprobar la calidad de los datos anuales extraídos de la instancia de Microsoft SQL Server para Norte Litoral

Es necesario hacer una validación de los datos antes de entrenar un modelo. La mayoría de los modelos entrenados son sensibles
a valores incorrectos ( valores nulos, fallos de medición y/o otros problemas ). Estos valores pueden ser interpretados de manera incorrecta por el modelo.

Proceso básico de validación de datos:

- 1)  Test Valores Nulos - Comprobar que se dispone de datos para todas las fechas y segmentos
        1.1) Comprobar que no hay nulos
        1.2) Comprobar que se dispone del rango de fechas solicitado
- 2)  Test Duplicados - Comprobar si existen filas duplicadas y quedarse con el primero
- 4)  Test de Coherencia de Datos - Verificar que no existen saltos significativos entre intervalos consecutivos de datos
        2.1) Calculo de autocorrelaciones parciales de datos 
        2.2) Establecimiento de un umbral significativo para descarte de datos 
        

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Test Cargando por Meses

In [3]:
enero = pd.read_csv('C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\monthly\\Datos_Anual\\01-18.csv', delimiter=";")

In [4]:
enero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59540 entries, 0 to 59539
Data columns (total 32 columns):
ID_SEGMENTO           59540 non-null int64
COD_LABORALIDAD       59540 non-null int64
FECHA                 59540 non-null object
MES                   59540 non-null int64
HORA                  59540 non-null int64
MINUTO                59540 non-null int64
CANTIDAD_PREC         59540 non-null object
TOTAL_VEHICULOS       59540 non-null int64
TOTAL_VEHICULOS_1     59540 non-null int64
TOTAL_VEHICULOS_2     59540 non-null int64
TOTAL_VEHICULOS_3     59540 non-null int64
TOTAL_VEHICULOS_4     59540 non-null int64
TOTAL_VEHICULOS_5     59540 non-null int64
TOTAL_VEHICULOS_6     59540 non-null int64
TOTAL_VEHICULOS_7     59540 non-null int64
TOTAL_VEHICULOS_8     59540 non-null int64
TOTAL_VEHICULOS_9     59540 non-null int64
TOTAL_VEHICULOS_10    59540 non-null int64
TOTAL_VEHICULOS_11    59540 non-null int64
TOTAL_VEHICULOS_12    59540 non-null int64
TOTAL_VEHICULOS_13    59540 n

In [27]:
from calendar import monthrange

for month_i in range(1,13):
    print("----> Executing tests for Month: " + str(month_i))
    month = pd.read_csv('C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\monthly\\Datos_Anual\\' + "{:02d}".format(month_i) + '-18.csv', delimiter=";")

    # Test 0 - check that all segments are available
    print("Test 0 - check that all segments are available")
    if len(month.ID_SEGMENTO.unique()) == 20:
        print("All segments are available")

    print("Test 1 - Missing values")

    for seg_i in enero.ID_SEGMENTO.unique():

        month_seg = month.loc[month.ID_SEGMENTO == seg_i]
        case_results = set()
        case_results.add(len(pd.to_datetime(month_seg.FECHA).dt.day.unique()) == monthrange(2018, month_i)[1])
        case_results.add(len(pd.to_datetime(month_seg.FECHA).dt.hour.unique()) == 24)
        case_results.add(len(pd.to_datetime(month_seg.FECHA).dt.minute.unique()) == 60/15)

    if case_results  == {True}:
        print("No missing values in the dataset")

    print("Test 2 - Duplicates")
    case_results = set()
    case_results.add(len(month.MES.unique()) == 1) 
    case_results.add(len(month[month.duplicated(['FECHA', 'ID_SEGMENTO'], keep='first')].index)==0)

    if case_results  == {True}:
        print("No duplicates values in the dataset")
    else: # Apply correction rules
        month = month.loc[month.MES==month_i] # nos quedamos solo con el mes que queremos
        month = month.drop_duplicates(['FECHA', 'ID_SEGMENTO'], keep='first')

    print("Saving results...")
    month.to_csv('C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\datos_validados\\' + "{:02d}".format(month_i) + '-18.csv', sep=';', index=False)

----> Executing tests for Month: 1
Test 0 - check that all segments are available
All segments are available
Test 1 - Missing values
No missing values in the dataset
Test 2 - Duplicates
Saving results...
----> Executing tests for Month: 2
Test 0 - check that all segments are available
All segments are available
Test 1 - Missing values
No missing values in the dataset
Test 2 - Duplicates
Saving results...
----> Executing tests for Month: 3
Test 0 - check that all segments are available
All segments are available
Test 1 - Missing values
No missing values in the dataset
Test 2 - Duplicates
Saving results...
----> Executing tests for Month: 4
Test 0 - check that all segments are available
All segments are available
Test 1 - Missing values
No missing values in the dataset
Test 2 - Duplicates
Saving results...
----> Executing tests for Month: 5
Test 0 - check that all segments are available
All segments are available
Test 1 - Missing values
No missing values in the dataset
Test 2 - Duplicate

In [28]:
import findspark
import pandas as pd
findspark.init()
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [37]:
data_input = sqlContext.read.format("com.databricks.spark.csv").options( header = True, inferSchema = True, sep=';',  line_terminator='\n').load( path='C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\input.csv')
df_anual_data = sqlContext.read.format("com.databricks.spark.csv").options( header = True, sep=';',  line_terminator='\n').schema(data_input.schema).load("C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\datos_validados\\*.csv") 

In [38]:
df = df_anual_data.toPandas()

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700800 entries, 0 to 700799
Data columns (total 32 columns):
ID_SEGMENT            691668 non-null float64
COD_LABORALIDAD       691668 non-null float64
FECHA                 691668 non-null datetime64[ns]
MES                   691668 non-null float64
HORA                  691668 non-null float64
MINUTO                691668 non-null float64
CANTIDAD_PREC         691668 non-null float64
TOTAL_VEHICULOS       691668 non-null float64
TOTAL_VEHICULOS_1     691668 non-null float64
TOTAL_VEHICULOS_2     691668 non-null float64
TOTAL_VEHICULOS_3     691668 non-null float64
TOTAL_VEHICULOS_4     691668 non-null float64
TOTAL_VEHICULOS_5     691668 non-null float64
TOTAL_VEHICULOS_6     691668 non-null float64
TOTAL_VEHICULOS_7     691668 non-null float64
TOTAL_VEHICULOS_8     691668 non-null float64
TOTAL_VEHICULOS_9     691668 non-null float64
TOTAL_VEHICULOS_10    691668 non-null float64
TOTAL_VEHICULOS_11    691668 non-null float64
TOTAL_VE

In [41]:
df = df.rename(columns={'ID_SEGMENT': 'ID_SEGMENTO'})

In [42]:
for month_i in range(1,13):
    print("----> Executing tests for Month: " + str(month_i))
    month = df.loc[df.ID_SEGMENTO == month_i]
    
    # Test 0 - check that all segments are available
    print("Test 0 - check that all segments are available")
    if len(month.ID_SEGMENTO.unique()) == 20:
        print("All segments are available")

    print("Test 1 - Missing values")

    for seg_i in enero.ID_SEGMENTO.unique():

        month_seg = month.loc[month.ID_SEGMENTO == seg_i]
        case_results = set()
        case_results.add(len(pd.to_datetime(month_seg.FECHA).dt.day.unique()) == monthrange(2018, month_i)[1])
        case_results.add(len(pd.to_datetime(month_seg.FECHA).dt.hour.unique()) == 24)
        case_results.add(len(pd.to_datetime(month_seg.FECHA).dt.minute.unique()) == 60/15)

    if case_results  == {True}:
        print("No missing values in the dataset")

    print("Test 2 - Duplicates")
    case_results = set()
    case_results.add(len(month.MES.unique()) == 1) 
    case_results.add(len(month[month.duplicated(['FECHA', 'ID_SEGMENTO'], keep='first')].index)==0)

    if case_results  == {True}:
        print("No duplicates values in the dataset")
    else: # Apply correction rules
        month = month.loc[month.MES==month_i] # nos quedamos solo con el mes que queremos
        month = month.drop_duplicates(['FECHA', 'ID_SEGMENTO'], keep='first')

print("Executed all tests")

----> Executing tests for Month: 1
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 2
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 3
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 4
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 5
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 6
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 7
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Duplicates
----> Executing tests for Month: 8
Test 0 - check that all segments are available
Test 1 - Missing values
Test 2 - Dup

In [43]:
df.to_csv('C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\datos_validados\\anual_data_18.csv', delimiter=";")

### 1)  Test Valores Nulos - Comprobar que se dispone de datos para todas las fechas y segmentos

In [4]:
anual_data = pd.read_csv('C:\\Users\\yhoz\\Documents\\dataanalytics.predictive\\data\\datos_2018_gen_modelos\\anual_data_2018.csv', delimiter=";")

c:\users\yhoz\.conda\envs\predictive.analytics\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
anual_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701045 entries, 0 to 701044
Data columns (total 32 columns):
ID_SEGMENTO           701045 non-null object
COD_LABORALIDAD       701045 non-null object
FECHA                 701045 non-null object
MES                   701045 non-null object
HORA                  701045 non-null object
MINUTO                701045 non-null object
CANTIDAD_PREC         701045 non-null object
TOTAL_VEHICULOS       701045 non-null object
TOTAL_VEHICULOS_1     701045 non-null object
TOTAL_VEHICULOS_2     701045 non-null object
TOTAL_VEHICULOS_3     701045 non-null object
TOTAL_VEHICULOS_4     701045 non-null object
TOTAL_VEHICULOS_5     701045 non-null object
TOTAL_VEHICULOS_6     701045 non-null object
TOTAL_VEHICULOS_7     701045 non-null object
TOTAL_VEHICULOS_8     701045 non-null object
TOTAL_VEHICULOS_9     701045 non-null object
TOTAL_VEHICULOS_10    701045 non-null object
TOTAL_VEHICULOS_11    701045 non-null object
TOTAL_VEHICULOS_12    701045 non-n

In [6]:
anual_data.isnull().any() # sum() to count them

ID_SEGMENTO           False
COD_LABORALIDAD       False
FECHA                 False
MES                   False
HORA                  False
MINUTO                False
CANTIDAD_PREC         False
TOTAL_VEHICULOS       False
TOTAL_VEHICULOS_1     False
TOTAL_VEHICULOS_2     False
TOTAL_VEHICULOS_3     False
TOTAL_VEHICULOS_4     False
TOTAL_VEHICULOS_5     False
TOTAL_VEHICULOS_6     False
TOTAL_VEHICULOS_7     False
TOTAL_VEHICULOS_8     False
TOTAL_VEHICULOS_9     False
TOTAL_VEHICULOS_10    False
TOTAL_VEHICULOS_11    False
TOTAL_VEHICULOS_12    False
TOTAL_VEHICULOS_13    False
TOTAL_VEHICULOS_14    False
TOTAL_VEHICULOS_15    False
TOTAL_VEHICULOS_16    False
TOTAL_VEHICULOS_17    False
TOTAL_VEHICULOS_18    False
TOTAL_VEHICULOS_19    False
TOTAL_VEHICULOS_20    False
TOTAL_VEHICULOS_21    False
TOTAL_VEHICULOS_22    False
TOTAL_VEHICULOS_23    False
TOTAL_VEHICULOS_24    False
dtype: bool

In [8]:
# Aproximacion estimada del numero de filas que deberia contener el dataset
365*24*(60/15)*20

700800.0

In [10]:
duplicates = anual_data[anual_data.duplicated(['FECHA', 'ID_SEGMENTO'], keep='first')]

In [12]:
duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 113321 to 641524
Data columns (total 32 columns):
ID_SEGMENTO           179 non-null object
COD_LABORALIDAD       179 non-null object
FECHA                 179 non-null object
MES                   179 non-null object
HORA                  179 non-null object
MINUTO                179 non-null object
CANTIDAD_PREC         179 non-null object
TOTAL_VEHICULOS       179 non-null object
TOTAL_VEHICULOS_1     179 non-null object
TOTAL_VEHICULOS_2     179 non-null object
TOTAL_VEHICULOS_3     179 non-null object
TOTAL_VEHICULOS_4     179 non-null object
TOTAL_VEHICULOS_5     179 non-null object
TOTAL_VEHICULOS_6     179 non-null object
TOTAL_VEHICULOS_7     179 non-null object
TOTAL_VEHICULOS_8     179 non-null object
TOTAL_VEHICULOS_9     179 non-null object
TOTAL_VEHICULOS_10    179 non-null object
TOTAL_VEHICULOS_11    179 non-null object
TOTAL_VEHICULOS_12    179 non-null object
TOTAL_VEHICULOS_13    179 non-null object
TOTAL

In [15]:
anual_data_no_duplicates = anual_data.drop_duplicates(['FECHA', 'ID_SEGMENTO'], keep='first')

In [16]:
anual_data_no_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700866 entries, 0 to 701044
Data columns (total 32 columns):
ID_SEGMENTO           700866 non-null object
COD_LABORALIDAD       700866 non-null object
FECHA                 700866 non-null object
MES                   700866 non-null object
HORA                  700866 non-null object
MINUTO                700866 non-null object
CANTIDAD_PREC         700866 non-null object
TOTAL_VEHICULOS       700866 non-null object
TOTAL_VEHICULOS_1     700866 non-null object
TOTAL_VEHICULOS_2     700866 non-null object
TOTAL_VEHICULOS_3     700866 non-null object
TOTAL_VEHICULOS_4     700866 non-null object
TOTAL_VEHICULOS_5     700866 non-null object
TOTAL_VEHICULOS_6     700866 non-null object
TOTAL_VEHICULOS_7     700866 non-null object
TOTAL_VEHICULOS_8     700866 non-null object
TOTAL_VEHICULOS_9     700866 non-null object
TOTAL_VEHICULOS_10    700866 non-null object
TOTAL_VEHICULOS_11    700866 non-null object
TOTAL_VEHICULOS_12    700866 non-n

In [17]:
700866 + 179

701045

### 1.2) Comprobar que se dispone del rango de fechas solicitado

In [19]:
anual_data_no_duplicates.MES.unique()

array([1, 2, '1', 'MES', '2', '3', 3, '4', 4, '5', 5, '6', 6, 7, 8, 9, 10,
       11, 12], dtype=object)